In [1]:
import pandas as pd
import re
import numpy as np
import praw
import datetime
import nltk
from nltk.corpus import stopwords

In [2]:
# I built these functions with help from 
# I built these funcitons with help from https://medium.com/swlh/scraping-reddit-using-python-57e61e322486

In [3]:
# Acessing the reddit api

reddit = praw.Reddit(client_id="mTiRlJ3LU5aM8w",                   # reddit client id
                     client_secret="cJCF5uuUynS9cRI2s-_DAvr-9ZQ",  # reddit client secret
                     user_agent="YouCanDoIt",                      # reddit user agent name
                     username = "joeynavarro-io",                  # reddit username
                     password = "$$06082006aC")                    # reddit password

In [4]:

def hot_sub_titles(topic, posts_to_get):
        # get 10 hot posts from the MachineLearning subreddit
        hot_posts = reddit.subreddit(topic).hot(limit= posts_to_get)
        for post in hot_posts:
            print(post.title)

In [5]:

def scrubber(subreddit_dictionary):    
    
    
        def get_clean_export(topic, posts_to_get):
            
            #function to change reddit website time format to datetime
            def convert_time(input):
                times = []
    
                for time in input:
        
                    conversion =  datetime.datetime.fromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
                    times.append(conversion)
                return times
        
            #obtain the reddits and put them in a dataframe
            posts = []
            ml_subreddit = reddit.subreddit(topic)

            for post in ml_subreddit.hot(limit = posts_to_get):
    
                posts.append([post.title, post.score, post.id, post.subreddit, 
                  post.url, post.num_comments, post.selftext, post.created])
    
            posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 
                                    'url', 'num_comments', 'body', 'created'])
            
            #prints the original shape of the posts dataframe
            print(f'{posts.shape} is the shape of the ' + topic + ' subreddit dataframe after scraping.')
            
            #format the dataframe the way I want
            posts['time_created'] = convert_time(posts['created'])
            posts.drop('created', axis=1, inplace=True)
            
            #remove links
            #got help from https://stackoverflow.com/questions/45395676/remove-a-url-row-by-row-from-a-large-set-of-text-in-python-panda-dataframe
            posts['body'] = posts['body'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
            posts['body'] = posts['body'].str.rstrip('.!? \n\t')
            posts['body'] = posts['body'].replace('[', '').replace(']', '')
            posts['body'] = posts['body'].replace('(', '').replace(')', '')
            posts['body'] = posts['body'].str.replace(r'\W', ' ')
            
            #remove the empty body fields
           
            posts = posts[(posts.body != '') & (posts.body != ' ') & (posts.body != 'Title') & (posts.body != np.nan)].reset_index()
           
            
            posts['title'] = posts['title'].str.replace(r'\W', ' ')
            
            #change subreddit name to topic
            posts['topic'] = posts['subreddit']
            
            #change column order
            posts = posts[['topic', 'time_created', 'title', 'id', 'score', 'num_comments', 'body','url']]
            
            #add prefix to columns
            posts = posts.add_prefix('subreddit_')
            
            #prints the shape of posts dataframe after removing empty subreddit_body
            print(f'>{posts.shape} is the shape of the ' + topic + ' subreddit dataframe after removing subreddit_body NaNs.')
            
            #turn dataframe into csv
            return posts.to_csv(f'./csv_scrapes/{topic}_subreddit.csv', index = False)
        
        for key, value in subreddit_dictionary.items():
    
            get_clean_export(key, value)

In [6]:
wanted_subreddits = {
    'philosophy' : 1000,
    'rant' : 1000,
    'Showerthoughts' : 1000,
    '80sRock': 1000,
    'dadjokes': 1000,
    'history' : 1000,
    'Electricity':1000
    
    
}

scrubber(wanted_subreddits)

(704, 8) is the shape of the philosophy subreddit dataframe after scraping.
>(40, 8) is the shape of the philosophy subreddit dataframe after removing subreddit_body NaNs.
(673, 8) is the shape of the rant subreddit dataframe after scraping.
>(656, 8) is the shape of the rant subreddit dataframe after removing subreddit_body NaNs.
(501, 8) is the shape of the Showerthoughts subreddit dataframe after scraping.
>(32, 8) is the shape of the Showerthoughts subreddit dataframe after removing subreddit_body NaNs.
(113, 8) is the shape of the 80sRock subreddit dataframe after scraping.
>(7, 8) is the shape of the 80sRock subreddit dataframe after removing subreddit_body NaNs.
(845, 8) is the shape of the dadjokes subreddit dataframe after scraping.
>(841, 8) is the shape of the dadjokes subreddit dataframe after removing subreddit_body NaNs.
(992, 8) is the shape of the history subreddit dataframe after scraping.
>(912, 8) is the shape of the history subreddit dataframe after removing subredd